In [1]:
target = input("請輸入你想得到的網址:")
print("下載目標:", target)

請輸入你想得到的網址:https://www.youtube.com/watch?v=or5A_ZzB6YI&t=179s
下載目標: https://www.youtube.com/watch?v=or5A_ZzB6YI&t=179s


In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
from browsermobproxy import Server
from selenium.common.exceptions import NoSuchElementException

server = Server("./browsermob-proxy-2.1.4/bin/browsermob-proxy")
server.start()
proxy = server.create_proxy()

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--proxy-server={0}".format(proxy.proxy))
driver = webdriver.Chrome(executable_path="./chromedriver", chrome_options = chrome_options)


proxy.new_har("youtube")
driver.get(target)
proxy.wait_for_traffic_to_stop(1, 30000) 

video = None
audio = None
for ent in proxy.har['log']['entries']:
    url = ent["request"]["url"]
    if "mime=video" in url:
        video = url
    if "mime=audio" in url:
        audio = url

print("影片:", video)
print("聲音:", audio)

server.stop()
driver.close()

影片: https://r6---sn-ipoxu-un5d.googlevideo.com/videoplayback?pl=21&itag=242&mt=1537434366&ms=au%2Conr&ei=c2OjW-HkCJWW4gLGh6WYDQ&mn=sn-ipoxu-un5d%2Csn-oguelnle&mm=31%2C26&c=WEB&keepalive=yes&id=o-AALeVpByvqYY-NkvGYVjIa-QqDI9m2iGKVkgGfxYtZt0&ip=223.136.106.48&sparams=aitags%2Cclen%2Cdur%2Cei%2Cgir%2Cid%2Cinitcwndbps%2Cip%2Cipbits%2Citag%2Ckeepalive%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Crequiressl%2Csource%2Cexpire&mv=m&key=yt6&lmt=1532853110274894&dur=292.759&fvip=1&mime=video%2Fwebm&initcwndbps=1137500&ipbits=0&clen=6951915&aitags=133%2C134%2C135%2C136%2C137%2C160%2C242%2C243%2C244%2C247%2C248%2C278&expire=1537456083&requiressl=yes&signature=3F5E62E42C2FAC44D213189A910872EA9DBC09AA.2247F217795770ECC2E3B7CB99F3E9EF9297F02B&source=youtube&gir=yes&alr=yes&cpn=sL3a6ZGKUcTg2KmY&cver=2.20180913&range=1286287-2022904&rn=30&rbuf=27795
聲音: https://r6---sn-ipoxu-un5d.googlevideo.com/videoplayback?pl=21&itag=251&mt=1537434366&ms=au%2Conr&ei=c2OjW-HkCJWW4gLGh6WYDQ&mn=sn-ipoxu-un5d%2Csn-oguelnle&m

In [4]:
import re
from urllib.request import urlretrieve
    
video = re.sub("range=[0-9]+-[0-9]+", "", video)
urlretrieve(video, "./tempvideo.webm")
if not audio is None:
    audio = re.sub("range=[0-9]+-[0-9]+", "", audio)
    urlretrieve(audio, "./tempaudio.webm")
print("完整影片:", video)
print("完整聲音:", audio)

完整影片: https://r6---sn-ipoxu-un5d.googlevideo.com/videoplayback?pl=21&itag=242&mt=1537434366&ms=au%2Conr&ei=c2OjW-HkCJWW4gLGh6WYDQ&mn=sn-ipoxu-un5d%2Csn-oguelnle&mm=31%2C26&c=WEB&keepalive=yes&id=o-AALeVpByvqYY-NkvGYVjIa-QqDI9m2iGKVkgGfxYtZt0&ip=223.136.106.48&sparams=aitags%2Cclen%2Cdur%2Cei%2Cgir%2Cid%2Cinitcwndbps%2Cip%2Cipbits%2Citag%2Ckeepalive%2Clmt%2Cmime%2Cmm%2Cmn%2Cms%2Cmv%2Cpl%2Crequiressl%2Csource%2Cexpire&mv=m&key=yt6&lmt=1532853110274894&dur=292.759&fvip=1&mime=video%2Fwebm&initcwndbps=1137500&ipbits=0&clen=6951915&aitags=133%2C134%2C135%2C136%2C137%2C160%2C242%2C243%2C244%2C247%2C248%2C278&expire=1537456083&requiressl=yes&signature=3F5E62E42C2FAC44D213189A910872EA9DBC09AA.2247F217795770ECC2E3B7CB99F3E9EF9297F02B&source=youtube&gir=yes&alr=yes&cpn=sL3a6ZGKUcTg2KmY&cver=2.20180913&&rn=30&rbuf=27795
完整聲音: https://r6---sn-ipoxu-un5d.googlevideo.com/videoplayback?pl=21&itag=251&mt=1537434366&ms=au%2Conr&ei=c2OjW-HkCJWW4gLGh6WYDQ&mn=sn-ipoxu-un5d%2Csn-oguelnle&mm=31%2C26&c=WEB&k

In [5]:
import os
from subprocess import PIPE, Popen
import datetime

title = str(datetime.datetime.now())

if not audio is None:
    command = ["ffmpeg", "-i", "tempvideo.webm", "-i", "tempaudio.webm", title + ".mp4"]
else:
    command = ["ffmpeg", "-i", "tempvideo.webm", title + ".mp4"]

with Popen(command, stdout=PIPE, stderr=PIPE) as p:
    result = p.communicate()
    
if not audio is None:
    os.remove("tempvideo.webm")
    os.remove("tempaudio.webm")
else:
    os.remove("tempvideo.webm")
    
result

(b'',
 b"ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers\n  built with Apple LLVM version 8.0.0 (clang-800.0.42.1)\n  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.0.2 --enable-shared --enable-pthreads --enable-version3 --enable-hardcoded-tables --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-gpl --enable-libmp3lame --enable-libx264 --enable-libxvid --enable-opencl --enable-videotoolbox --disable-lzma\n  libavutil      56. 14.100 / 56. 14.100\n  libavcodec     58. 18.100 / 58. 18.100\n  libavformat    58. 12.100 / 58. 12.100\n  libavdevice    58.  3.100 / 58.  3.100\n  libavfilter     7. 16.100 /  7. 16.100\n  libavresample   4.  0.  0 /  4.  0.  0\n  libswscale      5.  1.100 /  5.  1.100\n  libswresample   3.  1.100 /  3.  1.100\n  libpostproc    55.  1.100 / 55.  1.100\nInput #0, matroska,webm, from 'tempvideo.webm':\n  Metadata:\n    encoder         : google/video-file\n  Duration: 00:04:52.76, start: 0.000000, bitrate: 189 kb/s\n    S